In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from symmetry_lens import *
from os.path import join
from os import makedirs
import numpy as np

2025-02-22 23:17:49.983448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-22 23:17:50.108711: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 23:17:50.137964: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 23:17:50.742690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:
MODEL_SAVING_DIR = "model_weights_sensitivity_analysis"
NUM_TRAINING_EPOCHS = 2500
BATCH_SIZE = 630
OUTPUT_REPRESENTATION = "natural"
SYNTHETIC_DATASET_FEATURES = [
    {
        "type": "gaussian",
        "scale_min": 0.2,
        "scale_max": 1.0,
        "amplitude_min": 0.5,
        "amplitude_max": 1.5
    }
]
NOISE_STD = 0.05
WAVEFORM_TIMESTEPS=7

In [ ]:
# Create the model to train.
model = create_model(zero_padding_size=WAVEFORM_TIMESTEPS,
                     use_zero_padding=True,
                     num_uniformity_scales=1,
                     conditional_probability_estimator_hidden_layer_size = 112)

# Create model and load weights.
x_init = np.random.normal(size=(BATCH_SIZE, WAVEFORM_TIMESTEPS, 1))
model = create_model(zero_padding_size=WAVEFORM_TIMESTEPS,
                     use_zero_padding=True,
                     conditional_probability_estimator_hidden_layer_size = 112,
                     num_uniformity_scales=1)
model.compile()
model(x_init)

In [ ]:
def _compute_cosine_similarity_to_unit_shift(learned_gen):
    rotate_1 = np.roll(np.eye(WAVEFORM_TIMESTEPS), shift=1, axis=1)
    rotate_2 = np.roll(np.eye(WAVEFORM_TIMESTEPS), shift=-1, axis=1)
    
    vl = np.reshape(learned_gen, newshape=[WAVEFORM_TIMESTEPS*WAVEFORM_TIMESTEPS])
    v1 = np.reshape(rotate_1, newshape=[WAVEFORM_TIMESTEPS*WAVEFORM_TIMESTEPS])
    v2 = np.reshape(rotate_2, newshape=[WAVEFORM_TIMESTEPS*WAVEFORM_TIMESTEPS])
    
    c1 = vl * v1 / (np.linalg.norm(vl) * np.linalg.norm(v1)) 
    c2 = vl * v2 / (np.linalg.norm(vl) * np.linalg.norm(v2))
    
    return np.maximum(c1, c2) 

In [ ]:
# Zeroth elements are default parameters.
estimator_lr = [2.5e-3, 2.0e-3, 3.0e-3]
model_lr = [2.5e-4, 2.0e-4, 3.0e-4]
lr_decay = [0.1, 0.08, 0.12]
alignment = [1., 0.8, 1.2]
uniformity = [1., 0.8, 1.2]
resolution = [1., 0.8, 1.2]
infomax = [1., 0.8, 1.2]

params = zip(estimator_lr, 
             model_lr, 
             lr_decay, 
             alignment, 
             uniformity, 
             resolution, 
             infomax)

hyperparams = {"estimator_lr":estimator_lr, 
               "model_lr":model_lr, 
               "lr_decay":lr_decay, 
               "alignment":alignment, 
               "uniformity":uniformity, 
               "resolution":resolution, 
               "infomax":infomax}

scores = {"estimator_lr":[], 
          "model_lr":[], 
          "lr_decay":[], 
          "alignment":[], 
          "uniformity":[], 
          "resolution":[], 
          "infomax":[]}

default_experiment = True

for key in list(hyperparams.keys()):
    # Set default parameter values.
    exp_hyperparams = {"estimator_lr":estimator_lr[0], 
                       "model_lr":model_lr[0],
                       "lr_decay":lr_decay[0],
                       "alignment": alignment[0],
                       "uniformity": uniformity[0],
                       "resolution": resolution[0],
                       "infomax": infomax[0]}
    
    # Currently, it should work but this code is not clean. I guess, I could clean this more.
    if default_experiment:
        vals = hyperparams[key][:1]
    else:
        vals = hyperparams[key][1:]
        
    for val in vals:
        exp_hyperparams[key] = val
        model_path = join(MODEL_SAVING_DIR, f"modified:{key}__value:{val}", f"ep{NUM_TRAINING_EPOCHS-1}.h5")
        
        # At this point the module is loaded. Now, what to do?
        model.load_weights(model_path)
        score = _compute_cosine_similarity_to_unit_shift(model.symmetry_generator)
        
        # If this was the first experiment, it's the default one so
        if default_experiment:
            for _key in list(hyperparams.keys()):
                scores[_key].append(score)
            
            default_experiment = False
        else:
            scores[key].append(score)        
            
        # Then, what could I use the scores for? Could calculate local sensitivity. For each of the difference. 
        #Afterwards, I could select random points and could compute the global one? I guess this could be rather 
        #difficult. Instead of this, ablation study and local sensitivity analysis must suffice. 
        # Actually, I have four gpus. Then? 8 experiments of this kind could be fit in. Right? However, 
        #should change it more? Not sure about what should be done at this point.